In [ ]:
from opentrons import robot, containers, instruments
import argparse
import sys

import sqlalchemy
from sqlalchemy import create_engine,Column,Integer,String,ForeignKey,Table,Text,inspect,desc
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker,relationship

import os
import re
import math
import glob
import json
import numpy as np
import pandas as pd
from datetime import datetime
import getch

from config import *
from db_config import *

## Generate the SQL Database from json files and plate maps
session = build_db()


In [ ]:
# test = session.query(Part).filter(Part.part_id == 'BBF10K_000402').first()
# print(test.part_id,test.fragments[0].wells[0].address)

# test = session.query(Part).filter(Part.part_id == 'BBF10K_000403').first()
# print(test.part_id,test.fragments[0].wells[0].address)

# for part in session.query(Part).join(Fragment,Part.fragments).join(Well,Fragment.wells)\
#             .filter(Fragment.fragment_name.like('BBF10K_00000%'))\
#             .filter(Well != None).order_by(Part.part_id):
#     print(part.part_id)

# for part in session.query(Part).filter(Part.part_id.like('BBF10K_0000%')).order_by(Part.part_id):
#     print(part.status)

# for part in session.query(Part):
#     no_build = False
#     for frag in part.fragments:
#         if not frag.wells:
#             no_build = True
#     if not no_build:
#         part.change_status('received')


# for part in session.query(Part).filter(Part.part_id.like('BBF10K_0000%')).order_by(Part.part_id):
#     print(part.part_id,part.status)

    




    

In [ ]:
## ============================================
## Take in required information
## ============================================

# Set starting paths
# BASE_PATH is defined in the config file
PIPELINE_PATH = BASE_PATH + "/pipeline"
BUILDS_PATH = BASE_PATH + "/builds"
DATA_PATH = BASE_PATH + "/data"

# # Load files
# parser = argparse.ArgumentParser(description="Resuspend a plate of DNA on an Opentrons OT-1 robot.")
# parser.add_argument('-r', '--run', required=False, action="store_true", help="Send commands to the robot and print command output.")
# args = parser.parse_args()

## ============================================
## ESTABLISH INITIAL FUNCTIONS
## ============================================
def change_height(container,target):
    '''Allows for real-time calibration of the p10 single height'''
    x = 0
    print("Change height - s-g:up h-l:down x:exit")
    while x == 0:
        c = getch.getch()
        if c == "s":
            p10.robot._driver.move(z=20,mode="relative")
        elif c == "d":
            p10.robot._driver.move(z=5,mode="relative")
        elif c == "f":
            p10.robot._driver.move(z=0.5,mode="relative")
        elif c == "g":
            p10.robot._driver.move(z=0.1,mode="relative")
        elif c == "h":
            p10.robot._driver.move(z=-0.1,mode="relative")
        elif c == "j":
            p10.robot._driver.move(z=-0.5,mode="relative")
        elif c == "k":
            p10.robot._driver.move(z=-5,mode="relative")
        elif c == "l":
            p10.robot._driver.move(z=-20,mode="relative")
        elif c == "x":
            x = 1
    p10s.calibrate_position((container,target.from_center(x=0, y=0, z=-1,reference=container)))

def well_addresses():
    '''Generates a list of well address A1-H12'''
    letter = ["A","B","C","D","E","F","G","H"]
    number = ["1","2","3","4","5","6","7","8","9","10","11","12"]
    target_well = []
    temp_well = 0
    for n in number:
        for l in letter:
            temp_well = l + n
            target_well.append(temp_well)
    return target_well

def change_plates(current_plates):
    '''Allows the user to swap plates in the middle of a protocol'''
    source_plates = {}
    plate_locations = list(zip(pd.unique(current_plates),SOURCE_SLOTS[:len(current_plates)]))
    print("plate_locations","\n", plate_locations)
    input("Switch out plates for those listed. Press enter when ready.")
    for plate, slot in plate_locations:
        source_plates[plate] = containers.load('96-flat', slot)
    return source_plates

def make_gg_rxns(num_rxns,rxn_vol):
    '''Calculates the amount of each reagent to add to reach the desired master mix'''
    cutsmart = 1 * num_rxns
    atp = 1 * num_rxns
    vector = 0.25 * num_rxns
    ligase = 0.5 * num_rxns
    enzyme = 0.25 * num_rxns
    water = (rxn_vol - ((cutsmart + atp + vector + ligase + enzyme)/num_rxns)) * num_rxns
    master_mix = pd.DataFrame(
        {'Component':['H2O','Cutsmart','ATP','Vector','T4 Ligase','Restriction Enzyme','Total'],
        'Amount':[water,cutsmart,atp,vector,ligase,enzyme,rxn_vol*num_rxns]},
        columns=["Component","Amount"]
    )
    return master_mix

# # Verify that the correct robot is being used
# if args.run:
#     robot_name = str(os.environ["ROBOT_DEV"][-5:])
#     robot_number = str(input("Run on this robot: {} ? 1-Yes, 2-No ".format(robot_name)))
#     if robot_number == "1":
#         print("Proceeding with run")
#     else:
#         sys.exit("Run roboswitch 'ROBOT_NAME' to change to the correct robot")

target_well = well_addresses()

In [ ]:
## =============================================
## CREATE LIST OF DESIRED GENES
## =============================================

max_rxns = 96
to_build = []
acceptable_status = ['received']
# to_build = [part for part in session.query(Part).filter(Part.status.in_(acceptable_status)).\
#                         order_by(Part.part_id).limit(max_rxns)]

## Pulls in parts that have the desirable status and then sorts them by the plates their fragments are in,
## sorts them by their plate numbers and returns the earliest set of parts
to_build = [part for part in session.query(Part).join(Fragment,Part.fragments).\
            join(Well,Fragment.wells).join(Plate,Well.plates).\
            filter(Part.status.in_(acceptable_status)).filter(Plate.plate_name != 'pSHPs0807B412037MU').order_by(Plate.id)]
to_build = to_build[:96]
print("to build",len(to_build))

# unique_plates = []
# for part in to_build:
#     for frag in part.fragments:
#         unique_plates.append(frag.wells[0].plates.plate_name)
        
# unique_plates = pd.Series(unique_plates).unique()
# # print(unique_plates)

# target_build = Build(to_build)
# session.add(target_build)

# Grabs the most recently entered build
# target_build = session.query(Build).order_by(desc(Build.id)).first()

# addresses = []
# for plate in target_build.plates:
#     print(plate.plate_name,plate.next_well)
#     addresses.append([{'plate':well.plates.plate_name,'well':well.address,'frags':len(well.parts.fragments)}\
#                       for well in plate.wells if len(well.parts.fragments) > 1])
# total_extra = 0
# for plates in addresses:
#     for transfer in plates:
#         print(transfer['plate'],transfer['well'],transfer['frags'])
#         total_extra += transfer['frags']
# print(total_extra*8)

# for well in session.query(Well).join(Part,Well.parts).join(Fragment,Part.fragments).filter(len(Fragment) > 1).limit(10):
#     print(well.parts.part_id,well.address)



# ## =============================================
# ## CREATE PLATE GROUPS
# ## =============================================
# # Currently available spots on the OT-one deck
# SOURCE_SLOTS = ['D2','D3','B2']

# group_plates = [unique_plates[n:n+len(SOURCE_SLOTS)] for n in range(0, len(unique_plates), len(SOURCE_SLOTS))]
# for num,group in enumerate(group_plates):
#     print("Group{}: {}".format(num+1,group))
    
# input("Press enter to continue")

# # Create a means to index to the plates
# plate_dict = dict(enumerate(unique_plates))    


In [ ]:
# Set the volume of master mix to use per reaction
master_volume = 8
# Set a multiplier to account for pipetting error and evaporation
extra_master = 1.3

need_extra = []
for plate in target_build.plates:
    # print(plate.plate_name,plate.next_well)
    need_extra.append([{'plate':well.plates.plate_name,'well':well.address,'frags':len(well.parts.fragments)}\
                      for well in plate.wells if len(well.parts.fragments) > 1])
total_extra = 0
for plates in need_extra:
    for transfer in plates:
        print(transfer['plate'],transfer['well'],transfer['frags'])
        total_extra += transfer['frags']
num_reactions = len(to_build)
master_reactions = (total_extra + num_reactions) * master_volume * extra_master
print(total_extra)

# Generate the dataframe to present the master mix composition
master_mix = make_gg_rxns(master_reactions,master_volume)
print("Use the table below to create the master mix")
print()
print(master_mix)
print()
input("Press enter to continue")


In [ ]:
## =============================================
## SETUP THE OT-1 DECK
## =============================================
# Specify the locations of each object on the deck
locations = {
            "tiprack-200" : "A3",
            "tiprack-10" : "E2",
            "tiprack-10s1" : "E3",
            "tiprack-10s2" : "E1",
            "trash" : "D1",
            "PCR-strip-tall" : "C3",
            "DEST_PLATE" : "C2",
            "Tube_rack" : "B1"
        }

# Make the dataframe to represent the OT-1 deck
deck = ['A1','B2','C3','D2','E1']
slots = pd.Series(deck)
columns = sorted(slots.str[0].unique())
rows = sorted(slots.str[1].unique(), reverse=True)
layout_table = pd.DataFrame(index=rows, columns=columns)
layout_table.fillna("---", inplace=True)

# Fill in the data frame with the locations
for obj in locations:
        layout_table.loc[locations[obj][1], locations[obj][0]] = obj

# Displays the required plate map and waits to proceed
print("\n Please arrange the items in the following configuration: \n")
print(layout_table,"\n")
# input("Press enter to continue")

## =============================================
## SETUP THE MASTER MIX
## =============================================
# Calculate how many reactions worth of MM to make
total_num = 0
for index, row in master.iterrows():
    rxn_needed = int(row['Fragments'])
    total_num += rxn_needed
num_wells = len(master)
print("wells",num_wells)
num_rows = num_wells // 8
print('rows',num_rows)

# Set a multiplier to account for pipetting error and evaporation
extra_master = 1.3
master_reactions = total_num * extra_master

# Set the volume of master mix to use per reaction
master_volume = 8

# Generate the dataframe to present the master mix composition
master_mix = make_gg_rxns(master_reactions,master_volume)
print("Use the table below to create the master mix")
print()
print(master_mix)
print()
# input("Press enter to continue")


In [2]:
import argparse
import sys

import sqlalchemy
from sqlalchemy import create_engine,Column,Integer,String,ForeignKey,Table,Text,inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker,relationship

import os
import re
import math
import glob
import json
import numpy as np
import pandas as pd
from datetime import datetime

from config import *
from db_config import *


# for instance in session.query(Part).join(Well,Part.wells).join(Plate,Wells.plates).filter(Plate.plate_type == 'syn_plate').filter(Well.volume == None).order_by(Plate.id):
#     print(instance.plate_name)

for instance in session.query(Plate):#.join(Well,Plate.wells).filter(Plate.plate_type == 'syn_plate').filter(Well.volume == None).order_by(Plate.id):
    print(instance.plate_name)
#     for well in instance.wells:
#         print(well.address,well.syn_yield,well.volume)

Assembly_0
Assembly_0
Assembly_0
Assembly_0
Assembly_0
Assembly_0
build000
build003
build004
build005
build006
build008
pSHPs0807B412037MU
pSHPs0807B412038MU
pSHPs0807B412039MU
pSHPs0807B412040MU
pSHPs0826B426849MU
pSHPs0826B426850MU
pSHPs1025B525648MU
pSHPs1106B253446MU
pSHPs1121B618499MU
pSHPs1121B618500MU
pSHPs1207B618708MU
pSHPs1212B325156MU
pSHPs1212B325157MU
pSHPs1212B325158MU
pSHPs0220B502370MU
pSHPs0220B292296MU
Assembly_0
build000
build003
build004
build005
build006
build008
build007
build000
build003
build004
build005
build006
build008
build007
build000
build003
build004
build005
build006
build008
build007
pSHPs0301B054801MU
pSHPs0325B569005MU
pSHPs0325B569008MU
pSHPs0325B569010MU


In [ ]:
# plate = session.query(Plate).filter(Plate.plate_name == 'pSHPs1207B618708MU').one()
plate = session.query(Plate).filter(Plate.plate_name == 'pSHPs0807B412037MU').one()

print(plate.plate_name,len(plate.wells))
for well in plate.wells:
    print(well.fragments.fragment_name,well.volume)

In [18]:
import sqlalchemy
from sqlalchemy import create_engine,Column,Integer,String,ForeignKey,Table,Text,inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker,relationship

import glob
import json
import pandas as pd
from config import *
from db_config import *

names = []
id_nums = []
for file in sorted(glob.glob("{}/data/*/*.json".format(BASE_PATH))):
    with open(file,"r") as json_file:
        data = json.load(json_file)
    names.append(data['info']['documentation']['gene_name'])
    id_nums.append(data['gene_id'])
dictionary = dict(zip(names,id_nums))

for file in glob.glob('{}/plate_maps/*.csv'.format(BASE_PATH)):
    plate_map = pd.read_csv(file)
    try:
        plate_map['Plate']
        version = 1
        plate_key = 'Plate'
        well_key = 'Well'
        name_key = 'customer_line_item_id'
        sequence_key = 'Insert Sequence'
    except:
        try:
            plate_map['Plate ID']
            version = 2
            plate_key = 'Plate ID'
            well_key = 'Well Location'
            name_key = 'Name'
            sequence_key = 'Insert sequence'
        except:
            print("Doesn't fit the standard formats")
    unique = plate_map[plate_key].unique()
    for plate in unique:
        current_plate_map = plate_map[plate_map[plate_key] == plate]
        current_plate = Plate('','syn_plate',plate_name=plate)
        session.add(current_plate)
        for index,row in current_plate_map.iterrows():
            if fragment_name[:-2] not in names and fragment_name[:-2] not in id_nums:
                not_found.append(fragment_name)
                print("Not found: ",not_found)
                continue
            current_frag = ''
            current_frag = session.query(Fragment).filter(Fragment.seq == insert).first()
            if current_frag:
                current_plate.add_item(current_frag,well_address,syn_yield=row['Yield (ng)'])

            
#             print(row[well_key],row[name_key])

    

A1 	MMSYN1_0002_1
B1 	MMSYN1_0821_1
C1 	MMSYN1_0329_1
D1 	MMSYN1_0027_1
E1 	MMSYN1_0378_1
F1 	MMSYN1_0515_1
G1 	MMSYN1_0800_1
H1 	MMSYN1_0380_1
A2 	MMSYN1_0402_1
B2 	MMSYN1_0908_1
C2 	MMSYN1_0328_1
D2 	MMSYN1_0227_1
E2 	MMSYN1_0545_1
F2 	MMSYN1_0545_2
G2 	MMSYN1_0403_1
H2 	MMSYN1_0512_1
A3 	MMSYN1_0004_1
B3 	MMSYN1_0616_1
C3 	MMSYN1_0348_1
D3 	MMSYN1_0641_1
E3 	MMSYN1_0200_1
F3 	MMSYN1_0660_1
G3 	MMSYN1_0365_1
H3 	MMSYN1_0117_1
A4 	MMSYN1_0613_1
B4 	MMSYN1_0666_1
C4 	MMSYN1_0795_1
D4 	MMSYN1_0198_1
E4 	MMSYN1_0026_1
F4 	MMSYN1_0804_2
G4 	MMSYN1_0813_1
H4 	MMSYN1_0840_1
A5 	MMSYN1_0600_1
B5 	MMSYN1_0600_2
C5 	MMSYN1_0381_1
D5 	MMSYN1_0834_1
E5 	MMSYN1_0653_1
F5 	MMSYN1_0695_1
G5 	MMSYN1_0695_2
H5 	MMSYN1_0387_1
A6 	MMSYN1_0637_1
B6 	MMSYN1_0670_1
C6 	MMSYN1_0665_1
D6 	MMSYN1_0796_1
E6 	MMSYN1_0007_1
F6 	MMSYN1_0007_2
G6 	MMSYN1_0229_1
H6 	MMSYN1_0617_1
A7 	MMSYN1_0151_1
B7 	MMSYN1_0655_1
C7 	MMSYN1_0297_1
D7 	MMSYN1_0297_2
E7 	MMSYN1_0886_1
F7 	MMSYN1_0363_1
G7 	MMSYN1_0727_1
H7 	MMSYN1

D1 f2f2
E1 p1
F1 p2
G1 p3f1
H1 p3f2
A2 c1f1
B2 c1f2
C2 c1f3
D2 c2f1
E2 c2f2
F2 ACYX2008404
G2 JSVC2033447
H2 RRID2024547
A3 MIKW2005234
B3 NJKC2059279
C3 FNXH2130139
D3 XMVD2010505
E3 ERXG2008196
F3 ctrB_yeast_1
G3 crtI_rice_1
H3 crtI_yeast_1
A4 crtO_rice_1
B4 crtO_yeast_1
C4 crtB-b_rice_1
D4 crtB-b_yeast_1
E4 Mcherry-Zebrafish_1
F4 gene_15693_1
G4 gene_15693_2
H4 gene_20107_1
A5 gene_20107_2
B5 gene_4461_1
C5 gene_4461_2
D5 gene_8853_1
E5 gene_8853_2
F5 gene_12442_1
G5 gene_12442_2
H5 gene_12924_1
A6 gene_12924_2
B6 gene_13895_1
C6 gene_13895_2
D6 gene_14034_1
E6 gene_14034_2
F6 gene_14606_1
G6 gene_14606_2
H6 gene_20356_1
A7 gene_20356_2
B7 gene_20542_1
C7 gene_20542_2
D7 gene_21311_1
E7 gene_21311_2
F7 gene_6507_1
G7 gene_6507_2
H7 gene_8284_1
A8 gene_8284_2
B8 gene_120_1
C8 gene_120_2
D8 gene_12508_1
E8 gene_12508_2
F8 gene_13533_1
G8 gene_13533_2
H8 gene_17059_1
A9 gene_17059_2
B9 gene_17650_1
C9 gene_17650_2
D9 gene_20377_1
E9 gene_20377_2
F9 gene_20403_1
G9 gene_20403_2
H9 gene_